In [2]:
import streamlit as st
from transformers import pipeline, set_seed
import os

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
DLL load failed while importing _pywrap_tf2: A dynamic link library (DLL) initialization routine failed.

In [ ]:
st.set_page_config(page_title="AI Dungeon Story Generator", layout="centered")

st.title("🪄 AI Dungeon Story Generator")
st.markdown(
    """
    Create interactive fantasy and mystery stories using generative AI.
    Enter your story prompt, choose a genre, and generate multiple story continuations.
    Save your favorite story as a text file!
    """
)


In [ ]:
# Genre selection
genre = st.selectbox("Select a genre:", options=["Fantasy", "Mystery", "Sci-Fi", "Horror", "Adventure", "Romance"])

In [ ]:
# User input prompt
user_prompt = st.text_area("Enter your story prompt here:", height=150)

num_return_sequences = st.slider("Number of story continuations to generate:", min_value=1, max_value=5, value=3)

temperature = st.slider("Creativity (temperature):", min_value=0.7, max_value=1.2, value=0.9, step=0.1)

generate_button = st.button("Generate Stories")

In [ ]:
@st.cache_resource(show_spinner=False)
def load_model():
    # Load text generation pipeline with GPT-Neo or GPT-2 large for better story output
    try:
        generator = pipeline('text-generation', model='EleutherAI/gpt-neo-125M')
    except Exception as e:
        st.error(f"Error loading model: {e}")
        return None
    return generator

generator = load_model()

In [ ]:
if generate_button:
    if not user_prompt.strip():
        st.warning("Please enter a prompt to generate stories.")
    elif generator is None:
        st.error("Model not loaded. Try restarting the app.")
    else:
        # Prepend genre info to the prompt for better contextual generation
        prompt = f"Genre: {genre}\nStory start: {user_prompt}\nContinuation:"
        st.info("Generating... this may take a few seconds depending on your hardware and internet connection.")

        with st.spinner("Generating story continuations..."):
            try:
                generated_stories = generator(
                    prompt,
                    max_length=250,
                    num_return_sequences=num_return_sequences,
                    temperature=temperature,
                    do_sample=True,
                    top_p=0.9,
                    pad_token_id=50256,
                )
            except Exception as e:
                st.error(f"Generation error: {e}")
                generated_stories = []

        if generated_stories:
            st.success(f"Generated {len(generated_stories)} story continuations!")
            choice = st.radio("Select your favorite story continuation:", options=list(range(len(generated_stories))), format_func=lambda x: f"Story #{x+1}")

            # Clean up the story text, removing prompt prefix
            def clean_story(text):
                # Remove the prompt prefix from generated text if present
                if "Continuation:" in text:
                    parts = text.split("Continuation:")
                    return parts[-1].strip()
                return text.strip()

            selected_story = clean_story(generated_stories[choice]['generated_text'])

            st.write("### 🌟 Your selected story continuation:")
            st.markdown(selected_story)

            # Button to download the story as a text file
            def get_download_link(story_text):
                import base64
                b64 = base64.b64encode(story_text.encode()).decode()
                href = f'<a href="data:file/txt;base64,{b64}" download="ai_dungeon_story.txt">📥 Download story as text file</a>'
                return href

            st.markdown(get_download_link(selected_story), unsafe_allow_html=True)
        else:
            st.warning("No stories were generated. Try again.")

st.markdown("---")
st.markdown("Made with ❤️ using [Streamlit](https://streamlit.io/) and [Hugging Face Transformers](https://huggingface.co/transformers/)")
